In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('../ums_viz.csv')
df_perf = pd.read_pickle('descriptions.pkl')
df_perf.columns.values[0] = 'perf_name'

df['perf_name'] = df['perf_name'].apply(lambda x: x.strip())
df_perf['perf_name'] = df_perf['perf_name'].apply(lambda x: x.strip())

df['per_seat'] = df['tck_amt']/df['num_seats']
df_group = df.groupby('perf_name')

# Add a whole bunch of new performance-level features

# mean ticket amount, number of seats, price per seat
df_tmp = df_group['tck_amt', 'num_seats', 'per_seat'].mean().reset_index()
df_tmp = df_tmp.rename(columns = {'tck_amt': 'mean_tck_amt', 'num_seats': 'mean_num_seats', 'per_seat': 'mean_per_seat'})

df_perf = pd.merge(df_perf, df_tmp, on='perf_name')

# max ticket amount, number of seats, price per seat
df_tmp = df_group['tck_amt', 'num_seats', 'per_seat'].max().reset_index()
df_tmp = df_tmp.rename(columns = {'tck_amt': 'max_tck_amt', 'num_seats': 'max_num_seats', 'per_seat': 'max_per_seat'})
df_perf = pd.merge(df_perf, df_tmp, on='perf_name')

# total number of seats
df_tmp = df_group['num_seats'].count().reset_index()
df_tmp = df_tmp.rename(columns = {'num_seats': 'count_tck_amt'})
df_perf = pd.merge(df_perf, df_tmp, on='perf_name')

print df_perf

                perf_name                                        description  \
0     The Infernal Comedy  John Malkovich makes his UMS debut portraying ...   
1        Yuja Wang, piano  Chinese pianist Yuja Wang combines the spontan...   
2     London Philharmonic  The London Philharmonic returns for its first ...   
3        Hamburg Symphony  In 1971, French composer Olivier Messiaen was ...   
4    Chicago Symphony Orc  Wind players of the Chicago Symphony come toge...   
5    San Francisco Sympho  Michael Tilson Thomas and the San Francisco Sy...   
6      Pavel Haas Quartet  “The world’s most exciting string quartet? Wel...   
7        Ballet Preljocaj  Few audience members in attendance at Ballet P...   
8             Joshua Bell  Often referred to as a “poet of the violin,” J...   
9          Murray Perahia  In the more than 35 years he has been performi...   
10    Mariinsky Orchestra  The Mariinsky Orchestra and Valery Gergiev ret...   
11   Detroit Symphony Orc  A concert cre

In [124]:
# Extract bigram matrix
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

vect = CountVectorizer(ngram_range=(1, 1), stop_words = ENGLISH_STOP_WORDS, min_df = 5)
X = vect.fit_transform(df_perf.description.values)

print X.shape

(143, 446)


In [125]:
# Latent Dirichlet Allocation

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=5)
y = lda.fit_transform(X)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print

print_top_words(lda, vect.get_feature_names(), 20)

df_perf['topic'] = np.argmax(y, 1)

Topic #0:
ballet dance company program century ums new performance set dancers movement william chicago contemporary director art years works world thing
Topic #1:
music new york jazz times voice musical best year heart album contemporary ums international known ensemble legendary singer debut traditions
Topic #2:
award winning director tour london theatre national high broadcasts presented theater definition michigan new partnership people history house play things
Topic #3:
music ums theatre national michigan london theater arbor ann american high new composers play partnership concert hall orchestra available broadcasts
Topic #4:
music quartet ums program performance debut work group world audiences concert string classical appearance years great returns new opera playing



In [126]:
df_perf.groupby('topic').describe()

count_tck_amt  max_num_seats  max_per_seat   max_tck_amt  \
topic                                                                   
0     count       8.000000       8.000000      8.000000      8.000000   
      mean      770.875000      17.875000     33.250000    365.500000   
      std       974.780479      14.681743     24.609812    405.904633   
      min        17.000000       3.000000      0.000000      0.000000   
      25%       121.750000       7.000000      9.000000     54.000000   
      50%       527.000000      13.000000     48.000000    292.000000   
      75%       905.000000      27.000000     51.000000    474.000000   
      max      2977.000000      42.000000     54.000000   1188.000000   
1     count      42.000000      42.000000     42.000000     42.000000   
      mean      688.095238      31.190476     74.809524   1044.266667   
      std       635.406987      36.518781    147.860389   1450.964274   
      min        19.000000       4.000000     15.000000     80.000000   
      25%       266.500000      10.000000     44.000000    273.000000   
      50%       460.000000      18.000000     48.000000    422.500000   
      75%       908.000000      35.500000     60.000000   1172.500000   
      max      3267.000000     177.000000   1000.000000   6619.800000   
2     count      10.000000      10.000000     10.000000     10.000000   
      mean      348.600000      20.600000     28.800000    412.900000   
      std       415.302621      26.230601     14.335659    522.357998   
      min        44.000000       4.000000     22.000000     88.000000   
      25%        75.250000       6.500000     22.000000    143.000000   
      50%       108.000000       9.000000     22.000000    198.000000   
      75%       431.500000      21.500000     22.000000    258.000000   
      max      1186.000000      90.000000     56.000000   1620.000000   
3     count      27.000000      27.000000     27.000000     27.000000   
      mean      698.814815      35.370370     46.888889    751.159259   
      std      1082.728108      74.682996     26.398329   1024.416139   
      min        22.000000       4.000000     18.000000     72.000000   
      25%       140.000000       6.500000     22.000000    157.000000   
      50%       370.000000      17.000000     40.000000    350.000000   
      75%       844.500000      26.500000     60.000000    882.300000   
      max      5540.000000     392.000000    100.000000   4039.200000   
4     count      56.000000      56.000000     56.000000     56.000000   
      mean      844.839286      25.339286     53.964286    890.891071   
      std       738.839453      26.723008     16.523223   1978.151389   
      min       113.000000       4.000000     22.000000    165.600000   
      25%       433.250000      10.000000     46.000000    294.000000   
      50%       636.000000      18.500000     48.000000    519.000000   
      75%      1019.250000      33.250000     60.000000    839.625000   
      max      4540.000000     187.000000    120.000000  14960.000000   

             mean_num_seats  mean_per_seat  mean_tck_amt  
topic                                                     
0     count        8.000000       8.000000      8.000000  
      mean         1.898167      19.689914     37.638334  
      std          0.147214      14.384974     27.816925  
      min          1.590909       0.000000      0.000000  
      25%          1.863387       6.193548     10.625806  
      50%          1.907226      27.895396     53.327766  
      75%          1.987730      29.236940     57.388637  
      max          2.058824      33.781414     63.442792  
1     count       42.000000      42.000000     42.000000  
      mean         2.056273      31.658722     64.767773  
      std          0.195936      10.066202     20.601540  
      min          1.776846      11.000000     22.422222  
      25%          1.895161      26.589861     54.732105  
      50%          2.059160      30.673727     63.160184  
  